In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
import pandas as pd

# Upload the CSV file
#uploaded = files.upload()

# Read the uploaded CSV file
data = pd.read_csv("/content/processed_data.csv")  # Use the name of your uploaded file
data.head()


,traffic_volume,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,year,month,day,hour,day_in_week,is_weekend
0,1462,No,243.39,No rain,No snow,Clear,Haze,haze,2016,12,18,8,6,0
1,1037,No,243.62,No rain,No snow,Clear,Haze,haze,2016,12,18,7,6,0
2,800,No,244.22,No rain,No snow,Clear,Clear,sky is clear,2016,12,18,6,6,0
3,354,No,244.82,No rain,No snow,Party Cloudy,Clouds,few clouds,2013,2,2,3,5,0
4,417,No,244.82,No rain,No snow,Party Cloudy,Clouds,few clouds,2013,2,2,4,5,0


In [ ]:
have_snow = data[data['snow_1h'] == 'Have snow']
no_snow = data[data['snow_1h'] != 'Have snow']

snow_train = have_snow.sample(frac=0.9, random_state=42)
snow_test = have_snow.drop(snow_train.index)

print(snow_test.count())
print(snow_train.count())

traffic_volume         6
holiday                6
temp                   6
rain_1h                6
snow_1h                6
clouds_all             6
weather_main           6
weather_description    6
year                   6
month                  6
day                    6
hour                   6
day_in_week            6
is_weekend             6
dtype: int64
traffic_volume         57
holiday                57
temp                   57
rain_1h                57
snow_1h                57
clouds_all             57
weather_main           57
weather_description    57
year                   57
month                  57
day                    57
hour                   57
day_in_week            57
is_weekend             57
dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming you want an 80/20 split, adjust test_size as needed
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

print(f"train dataset length: ", len(train_data))
print(f"test data length: ", len(test_data))

train dataset length:  38554
test data length:  9639


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# Selecting features and target
features = data.drop(columns=["traffic_volume"])
target = data["traffic_volume"]

# Preprocessing: Encoding categorical variables and scaling numerical ones
numerical_features = ["temp"]
categorical_features = ["holiday", "rain_1h", "snow_1h", "clouds_all", "weather_main"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),
        ("cat", OneHotEncoder(drop="first"), categorical_features),
    ]
)

# Reducing the dataset to a smaller sample for faster execution
sampled_data = data.sample(frac=0.1, random_state=42)
features_sampled = sampled_data.drop(columns=["traffic_volume"])
target_sampled = sampled_data["traffic_volume"]
X_train, X_test, y_train, y_test = train_test_split(features_sampled, target_sampled, test_size=0.2, random_state=42)

# Building a pipeline with preprocessing and SVM model with fixed parameters
svm_pipeline_fixed = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", SVR(kernel="rbf", C=10, gamma=0.1))
])

# Fitting the model with fixed parameters
svm_pipeline_fixed.fit(X_train, y_train)

# Making predictions on the test set
y_pred_fixed = svm_pipeline_fixed.predict(X_test)

# Evaluating the model
mse_fixed = mean_squared_error(y_test, y_pred_fixed)

mse_fixed


3817126.1693534977

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# Selecting features and target
features = data.drop(columns=["traffic_volume"])
target = data["traffic_volume"]

# Preprocessing: Encoding categorical variables and scaling numerical ones
numerical_features = ["temp"]
categorical_features = ["holiday", "rain_1h", "snow_1h", "clouds_all", "weather_main"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),
        ("cat", OneHotEncoder(drop="first"), categorical_features),
    ]
)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Building a pipeline with preprocessing and SVM model with fixed parameters
svm_pipeline_fixed = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", SVR(kernel="rbf", C=10, gamma=0.1))
])

# Fitting the model with fixed parameters
svm_pipeline_fixed.fit(X_train, y_train)

# Making predictions on the test set
y_pred_fixed = svm_pipeline_fixed.predict(X_test)

# Evaluating the model
mse_fixed = mean_squared_error(y_test, y_pred_fixed)

mse_fixed

3740618.623999217

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

# Selecting features and target
features = data.drop(columns=["traffic_volume"])
target = data["traffic_volume"]

# Preprocessing: Encoding categorical variables and scaling numerical ones
numerical_features = ["temp"]
categorical_features = ["holiday", "rain_1h", "snow_1h", "clouds_all", "weather_main"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),
        ("cat", OneHotEncoder(drop="first"), categorical_features),
    ]
)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Building a pipeline with preprocessing and SVM model with fixed parameters
svm_pipeline_fixed = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", SVR(kernel="rbf", C=10, gamma=0.1))
])

# Fitting the model with fixed parameters
svm_pipeline_fixed.fit(X_train, y_train)

# Making predictions on the test set
y_pred_fixed = svm_pipeline_fixed.predict(X_test)

# Evaluating the model
mse_fixed = mean_squared_error(y_test, y_pred_fixed)
r2_train = r2_score(y_train, svm_pipeline_fixed.predict(X_train))
rmse = sqrt(mse_fixed)

print(f"\n Best Hyperparameters: {{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}}")
print(f"\n R2 on Train-set: {r2_train}")
print(f"\n Root Mean Squared Error (RMSE) on Test Set: {rmse}")



 Best Hyperparameters: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}

 R2 on Train-set: 0.044653411389640185

 Root Mean Squared Error (RMSE) on Test Set: 1934.0678953954066


In [11]:
import joblib
def save_file(scaler, filename):
    joblib.dump(scaler, filename)
    return f"Saving {filename} successfully executed"

In [12]:
import joblib

def save_file(scaler, filename):
    joblib.dump(scaler, filename)
    return f"Saving {filename} successfully executed"

# Ví dụ:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
save_file(scaler, "SVM_model.joblib")


'Saving SVM_model.joblib successfully executed'

In [13]:
from google.colab import files

# Tải file về máy
files.download("SVM_model.joblib")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>